In [ ]:
import numpy as np
import os, shutil
from tqdm import tqdm_notebook
import pandas as pd
import open3d as o3d
import gc
import matplotlib.pyplot as plt
import copy

In [ ]:
%matplotlib notebook

In [ ]:
BASE_PATH = "/home/sabyasachi/Projects/ati/data/data/datasets/Carla/16beam-Data/small_map/validate_idea/dynamic"
BASE_RANGE_IMAGE_FOLDER = "_static_only_out_npy"
OVERLAP_RANGE_IMAGE_FOLDER = "corr_static_transform_stronger_out_npy"
OUTPUT_PCD_FOLDER = "corr_static_transform_stronger_filtered"
OUTPUT_NPY_FOLDER = "corr_static_transform_stronger_filtered_out_npy"

In [ ]:
# for MAIN_FOLDER in os.listdir(BASE_PATH):
MAIN_FOLDER = "24"
for _ in range(1):
    print("\n************* \n Subfolder: {}".format(MAIN_FOLDER))
    print("Setting paths")
    BASE_RANGE_IMAGE_PATH = os.path.join(BASE_PATH, MAIN_FOLDER, BASE_RANGE_IMAGE_FOLDER)
    OVERLAP_RANGE_IMAGE_PATH = os.path.join(BASE_PATH, MAIN_FOLDER, OVERLAP_RANGE_IMAGE_FOLDER)
    OUTPUT_PCD_PATH = os.path.join(BASE_PATH, MAIN_FOLDER, OUTPUT_PCD_FOLDER)
    if not os.path.exists(OUTPUT_PCD_PATH):
        os.makedirs(OUTPUT_PCD_PATH)
    else:
        shutil.rmtree(OUTPUT_PCD_PATH)
        os.makedirs(OUTPUT_PCD_PATH)
    OUTPUT_NPY_PATH = os.path.join(BASE_PATH, MAIN_FOLDER, OUTPUT_NPY_FOLDER)
    if not os.path.exists(OUTPUT_NPY_PATH):
        os.makedirs(OUTPUT_NPY_PATH)
    else:
        shutil.rmtree(OUTPUT_NPY_PATH)
        os.makedirs(OUTPUT_NPY_PATH)
    gc.collect()
    
    print("Loading npys")
    base_files = os.listdir(BASE_RANGE_IMAGE_PATH)
    overlap_files = os.listdir(OVERLAP_RANGE_IMAGE_PATH)

    base_file_path = os.path.join(BASE_RANGE_IMAGE_PATH, base_files[0])
    overlap_file_path = os.path.join(OVERLAP_RANGE_IMAGE_PATH, overlap_files[0])

    base_arr = np.load(base_file_path)
    overlap_arr = np.load(overlap_file_path)
    gc.collect()
    
    print("Start looping")
    new_base_list = []
    file_idx = 1

    for base_img, overlap_img in tqdm_notebook(zip(base_arr, overlap_arr), total=base_arr.shape[0]):
        base_set = set([(i,j) for i in range(base_img.shape[0]) \
                                 for j in range(base_img[i].shape[0]) \
                                     if base_img[i,j,0] == base_img[i,j,1] == base_img[i,j,2] == 0.0])
        overlap_set = set([(i,j) for i in range(overlap_img.shape[0]) \
                                 for j in range(overlap_img[i].shape[0]) \
                                     if overlap_img[i,j,0] != 0.0 and \
                                        overlap_img[i,j,1] != 0.0 and \
                                        overlap_img[i,j,2] != 0.0])
        fill_set = base_set.intersection(overlap_set)

        new_base_img = copy.deepcopy(base_img)
        for i, j in fill_set:
            for k in range(new_base_img.shape[-1]):
                new_base_img[i,j,k] = overlap_img[i,j,k]

        new_base_list.append(new_base_img)
        this_arr = np.reshape(new_base_img, (-1,4))[:,:3]
        this_arr = this_arr[~np.array([x == y == z == 0 for x,y,z in this_arr])]

        this_pcd = o3d.PointCloud()
        this_pcd.points = o3d.utility.Vector3dVector(this_arr)
        fname = str(file_idx) + ".ply"
        fpath = os.path.join(OUTPUT_PCD_PATH, fname)

        o3d.io.write_point_cloud(fpath, this_pcd, write_ascii=True)
        file_idx += 1
        gc.collect()
#         break
    print("Saving npy")
    npy_fname = MAIN_FOLDER + ".npy"
    npy_fpath = os.path.join(OUTPUT_NPY_PATH, npy_fname)
    np.array(new_base_list).dump(npy_fpath)
    gc.collect()
#     break

In [ ]:
base_arr[0] - overlap_arr[0]